In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 4.4 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=1e877d3d81d7398c9461091a8e8d24620ba3cba7f7555221cfc2517ca95197c0
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 989 kB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_UK   dataframe_csv_EU
 a0529-a0611_US   dataframe_csv_UK
 a0529.csv	  dataframe_result_csv
 a0530.csv	  dataframe_result_csv_EU
 a0531.csv	  dataframe_result_csv_UK
 a0601.csv	  Github
 a0602.csv	  heatmap529_UK.ipynb
 a0603.csv	  heatmap530.ipynb
 a0604.csv	  Heat_Map_Online_language.ipynb
 a0605.csv	  __MACOSX
 a0606.csv	  non_violant1.csv
 a0607.csv	  non_violant2.csv
 csv		  UK
 csv.zip	  violant.csv
 dataframe_csv	 '新建 Microsoft PowerPoint 演示文稿.pptx'


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_531 = 'UK/06_11.csv'

In [8]:
ct = pd.read_csv(ct_path_531)

In [9]:
ct.shape[1]

47

In [10]:
#columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,119550118426654_1144228372625485,Facebook,2020-06-11 23:59:26,2021-09-01 11:27:10,video,Peaceful Protests Spark in Wyandanch and Brent...,shadesoflongisland.com,Please take 8:46 to watch this film. DON’T LOO...,Take the time it took to kill George Floyd and...,1.0,https://shadesoflongisland.com/2020/06/01/peac...,https://www.facebook.com/groups/11955011842665...,0,-0.277778,1.0,6,6,1,0,0,0,0,0,0,2,5,2,2,2,0,0,6,1,0,2,12884237,Race Unity Group of Carbondale,NaN,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,NaN,NaN,False,NaN,en,0,12884237|1144228372625485
1,352050521888601_911053385988309,Facebook,2020-06-11 23:59:24,2021-07-26 15:24:48,link,"Perspective | Remember Neli Latson, the black ...",washingtonpost.com,Latson’s case shows that while some losses occ...,Neli Latson’s life changed in the same swift w...,1.0,https://www.washingtonpost.com/local/remember-...,https://www.facebook.com/groups/35205052188860...,0,-0.916667,1.0,0,0,0,0,0,0,1,0,0,0,2,1,3,1,1,2,1,1,0,1,15825951,The Truth!!!,NaN,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,NaN,NaN,False,NaN,en,0,15825951|911053385988309
2,179948425230_10163588345370231,Facebook,2020-06-11 23:59:20,2021-07-09 21:53:15,native_video,"Perspective | Remember Neli Latson, the black ...",washingtonpost.com,Latson’s case shows that while some losses occ...,Talking about next steps for justice live on @...,1.0,https://www.facebook.com/realsharpton/videos/2...,https://www.facebook.com/179948425230/posts/10...,597764,2.771739,2.0,291,90,52,71,2,1,1,2,0,18,106,29,20,18,2,4,3,2,0,5,48824,Reverend Al Sharpton,realsharpton,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,FOLLOW ME ON TWITTER! www.twitter.com/TheRevAl...,True,2009-05-26 00:56:53,en,0,48824|10163588345370231
3,245354138137_10157682143678138,Facebook,2020-06-11 23:59:18,2021-03-02 18:50:37,link,UUC Gathers Online,mailchi.mp,I write today with a heavy heart over the deat...,UUC Gathers Online - https://mailchi.mp/d91def...,2.0,https://mailchi.mp/d91def54f14b/8pcnwx8zix-421...,https://www.facebook.com/245354138137/posts/10...,374,-0.857143,1.0,1,0,0,0,0,0,0,0,0,0,2,2,2,1,0,0,0,0,0,0,223616,Unitarian Universalist Congregation in Stamfor...,realsharpton,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,We exist to build a community that deepens ind...,False,2010-01-14 16:49:39,en,0,223616|10157682143678138
4,250147468771235_982208315565143,Facebook,2020-06-11 23:59:17,2021-09-07 01:05:13,link,Biden says Floyd death having bigger global im...,apple.news,Former Vice President Joe Biden on Thursday sa...,Right when you think Joe Biden can't get anymo...,1.0,https://apple.news/AduJeEjRNQPOBcyKxx_62jg,https://www.facebook.com/groups/25014746877123...,876,-0.764706,1.0,0,1,3,0,0,0,0,0,0,0,2,2,4,1,2,2,2,2,0,1,10779034,Bronx Minds on Politics,realsharpton,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_group,US,We exist to build a community that deepens ind...,False,2010-01-14 16:49:39,en,0,10779034|982208315565143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15153,110323774042312_116443710096985,Facebook,2020-06-11 12:00:0

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,{},{}
1,{'Virginia': 1},{}
2,{},{}
3,{},{'Minneapolis': 1}
4,{},{}


In [24]:
ct.to_csv('a0611.csv')

In [25]:
ct=pd.read_csv('a0611.csv',lineterminator="\n")

In [26]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [27]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [28]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{},{},"{'Wyandanch': 1, 'Brentwood': 1, 'Long Island'..."
1,{'Virginia': 1},{},{}
3,{},{'Minneapolis': 1},{}
6,{},"{'Minneapolis': 1, 'USA': 1}",{}
7,{},{'Tacoma': 1},{'Seattle': 1}


In [29]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [30]:
ct_ner['NER-msg']

0                            []
1                    [Virginia]
3                            []
6                            []
7                            []
                  ...          
15151    [Minneapolis, Toronto]
15152                        []
15154                 [Houston]
15155                        []
15156               [Minnesota]
Name: NER-msg, Length: 10029, dtype: object

In [31]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
ct_ner['NER']

0          [Brentwood, Long Island, Wyandanch]
1                                   [Virginia]
3                                [Minneapolis]
6                           [USA, Minneapolis]
7                            [Tacoma, Seattle]
                         ...                  
15151      [Minneapolis, Toronto, Minneapolis]
15152                            [Minneapolis]
15154                                [Houston]
15155               [San Antonio, Minneapolis]
15156    [Minnesota, San Antonio, Minneapolis]
Name: NER, Length: 10029, dtype: object

In [33]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
ner_lst = ct_ner['NER'].values

In [36]:
ner_lst

array([list(['brentwood', 'long island', 'wyandanch']),
       list(['virginia']), list(['minneapolis']), ..., list(['houston']),
       list(['san antonio', 'minneapolis']),
       list(['minnesota', 'san antonio', 'minneapolis'])], dtype=object)

In [37]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [38]:
list

['brentwood',
 'long island',
 'wyandanch',
 'virginia',
 'minneapolis',
 'usa',
 'minneapolis',
 'tacoma',
 'seattle',
 'dallas',
 'texas',
 'usa',
 'south africa',
 'united states of america',
 'usa',
 'south africa',
 'united states of america',
 'usa',
 'se portland',
 'south africa',
 'burgerville',
 'united states of america',
 'fort worth',
 '|',
 'texas',
 'seattle',
 'seattle',
 'seattle',
 'minneapolis',
 'america"—just',
 'minneapolis',
 'denton',
 'washington',
 'u.s.',
 'us',
 'the united states',
 'us',
 'the united states',
 'newark',
 'new jersey’s',
 'n.j.',
 'newark',
 'new jersey’s',
 'florida',
 'n.j.',
 'newark',
 'new jersey’s',
 'n.j.',
 'australia',
 'the united states',
 'broome',
 'australia',
 'the united states',
 'broome',
 'florida',
 'minneapolis',
 'chicago',
 'u.s.',
 'chicago',
 'minneapolis',
 'louisville',
 'iowa',
 'md',
 'america',
 'minneapolis',
 'alaska',
 'alaska',
 'sylvania',
 'maplewood ave',
 'minneapolis',
 'elkhart',
 'elkhart',
 'minneap

In [39]:
c = collections.Counter()

In [40]:
dict_a=collections.Counter(list).most_common()

In [41]:
dict_a

[('minneapolis', 3206),
 ('america', 1210),
 ('us', 1102),
 ('u.s.', 741),
 ('the united states', 695),
 ('houston', 553),
 ('minnesota', 528),
 ('washington', 467),
 ('texas', 331),
 ('seattle', 261),
 ('chicago', 243),
 ('hennepin county', 238),
 ('new york', 231),
 ('los angeles', 218),
 ('virginia', 216),
 ('california', 186),
 ('new jersey', 173),
 ('uk', 172),
 ('richmond', 168),
 ('d.c.', 153),
 ('new york city', 153),
 ('nigeria', 147),
 ('china', 131),
 ('columbus', 130),
 ('boston', 130),
 ('usa', 128),
 ('connecticut', 127),
 ('massachusetts', 127),
 ('florida', 126),
 ('dallas', 124),
 ('canada', 120),
 ('new hampshire', 119),
 ('maine', 117),
 ('australia', 116),
 ('alabama', 114),
 ('germany', 109),
 ('philadelphia', 107),
 ('minn.', 102),
 ('nyc', 98),
 ('israel', 98),
 ('rhode', 97),
 ('london', 94),
 ('ghana', 90),
 ('buffalo', 89),
 ('dc', 89),
 ('wisconsin', 89),
 ('maryland', 84),
 ('france', 82),
 ('the united states of america', 81),
 ('georgia', 78),
 ('britain',

In [42]:
dataframe531=pd.DataFrame(dict_a)

In [43]:
len(dict_a)

2359

In [44]:
dataframe531.to_csv('dataframe0611_UK.csv')